In [3]:
from subprocess import check_output
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

warnings.filterwarnings('ignore')
num_classes = 10
epochs = 20

train_df = pd.read_csv('./input/fashion-mnist_train.csv', sep=',')
test_df = pd.read_csv('./input/fashion-mnist_test.csv', sep = ',')

# print(check_output(["ls", "./input"]).decode("utf8"))
print(train_df.head())
print(train_df.shape)


fashion-mnist_test.csv
fashion-mnist_train.csv
t10k-images-idx3-ubyte
t10k-labels-idx1-ubyte
train-images-idx3-ubyte
train-labels-idx1-ubyte

   label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      2       0       0       0       0       0       0       0       0   
1      9       0       0       0       0       0       0       0       0   
2      6       0       0       0       0       0       0       0       5   
3      0       0       0       0       1       2       0       0       0   
4      3       0       0       0       0       0       0       0       0   

   pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0       0  ...         0         0         0         0         0         0   
1       0  ...         0         0         0         0         0         0   
2       0  ...         0         0         0        30        43         0   
3       0  ...         3         0         0         0         0         1   
4       0  

In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, Flatten, Dense
from keras.models import Model
from keras.utils import to_categorical

class_names = ['T_shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

x_train = train_df.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y_train = to_categorical(train_df.iloc[:, 0].values, 10)
x_test = test_df.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y_test = to_categorical(test_df.iloc[:, 0].values, 10)

x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.2, random_state = 12345)


# Definicja bloku resztkowego
def residual_block(inputs, filters):
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Add()([x, inputs])
    x = Activation('relu')(x)
    return x

inputs = Input(shape=(28, 28, 1))
x = Conv2D(64, kernel_size=(3, 3), padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = residual_block(x, 64)
x = residual_block(x, 64)

x = Conv2D(128, kernel_size=(3, 3), padding='same', strides=(2, 2))(x)
x = residual_block(x, 128)
x = residual_block(x, 128)

x = Conv2D(256, kernel_size=(3, 3), padding='same', strides=(2, 2))(x)
x = residual_block(x, 256)
x = residual_block(x, 256)

x = Flatten()(x)
x = Dense(512, activation='relu')(x)
outputs = Dense(10, activation='softmax')(x)

model = Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=64, epochs=10, validation_data=(x_validate, y_validate))

loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('===========================')
print('Test loss:', loss)
print('Test accuracy:', accuracy)
print('===========================')


Epoch 1/10


2023-02-23 09:56:39.271242: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


750/750 [==============================] - ETA: 0s - loss: 0.8652 - accuracy: 0.7341

2023-02-23 09:58:35.606487: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


750/750 [==============================] - 126s 167ms/step - loss: 0.8652 - accuracy: 0.7341 - val_loss: 0.3446 - val_accuracy: 0.8766
Epoch 2/10
750/750 [==============================] - 126s 168ms/step - loss: 0.4167 - accuracy: 0.8587 - val_loss: 0.3088 - val_accuracy: 0.8872
Epoch 3/10
750/750 [==============================] - 122s 162ms/step - loss: 0.2997 - accuracy: 0.8923 - val_loss: 0.2698 - val_accuracy: 0.8994
Epoch 4/10
750/750 [==============================] - 121s 161ms/step - loss: 0.2608 - accuracy: 0.9049 - val_loss: 0.2815 - val_accuracy: 0.8977
Epoch 5/10
750/750 [==============================] - 124s 165ms/step - loss: 0.2285 - accuracy: 0.9168 - val_loss: 0.2588 - val_accuracy: 0.9075
Epoch 6/10
750/750 [==============================] - 127s 170ms/step - loss: 0.2037 - accuracy: 0.9256 - val_loss: 0.2438 - val_accuracy: 0.9116
Epoch 7/10
750/750 [==============================] - 129s 172ms/step - loss: 0.1894 - accuracy: 0.9313 - val_loss: 0.2671 - val_accura

In [25]:
from sklearn.metrics import classification_report
import tensorflow as tf

# , 'pool_size': 2 # down sampling the output instead of 28*28 it is 14*14
params = [
    # # -0.01
    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.0005
    #     , 'batch_size': 4096
    #     , 'epochs': 75},
    # + 0.12

    {   'model': Sequential([
            Conv2D(filters=16, kernel_size=4, activation='relu', input_shape = image_shape),
            Dropout(0.2),
            MaxPooling2D(pool_size=2),
            Dropout(0.2), Flatten(),
            Dense(16, activation='relu'),
            Dense(10, activation = 'softmax')])
        , 'adam_lr': 0.0025
        , 'batch_size': 64
        , 'epochs': 30},

    # {   'model': Sequential([
    #         Conv2D(filters=16, kernel_size=4, activation='relu', input_shape = image_shape),
    #         Dropout(0.2),
    #         MaxPooling2D(pool_size=2),
    #         Dropout(0.2), Flatten(),
    #         Dense(16, activation='relu'),
    #         Dense(10, activation = 'softmax')])
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 128
    #     , 'epochs': 30},


    # {   'model': Sequential([
    #         Conv2D(filters=16, kernel_size=4, activation='relu', input_shape = image_shape),
    #         Dropout(0.2),
    #         MaxPooling2D(pool_size=2),
    #         Dropout(0.2), Flatten(),
    #         Dense(32, activation='relu'),
    #         Dense(10, activation = 'softmax')])
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 1024
    #     , 'epochs': 30},

    # {   'model': Sequential([
    #         Conv2D(filters=16, kernel_size=4, activation='relu', input_shape = image_shape),
    #         MaxPooling2D(pool_size=2), Dropout(0.2), Flatten(),
    #         Dense(32, activation='relu'),
    #         Dense(10, activation = 'softmax')])
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 1024
    #     , 'epochs': 30},

    # {     'filters': 32
    #     , 'kernel_size': 4
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 1024
    #     , 'epochs': 30},

    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 1024
    #     , 'epochs': 30},
    #
    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 2048
    #     , 'epochs': 30},

    # -.01
    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.0025
    #     , 'batch_size': 4096
    #     , 'epochs': 30},

    # -.01
    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.002
    #     , 'batch_size': 4096
    #     , 'epochs': 30},
    # -.01
    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 24
    #     , 'adam_lr': 0.001
    #     , 'batch_size': 4096
    #     , 'epochs': 35},
    # {     'filters': 32
    #     , 'kernel_size': 3
    #     , 'pool_size': 2
    #     , 'dropout': 0.2
    #     , 'dense_units': 32
    #     , 'adam_lr': 0.001
    #     , 'batch_size': 4096
    #     , 'epochs': 35},
]

for p in params:
    print('>>> Params: ', p)
    cnn_model = p['model']
    cnn_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=p['adam_lr']), metrics =['accuracy'])
    history = cnn_model.fit(
        x_train,
        y_train,
        batch_size=p['batch_size'],
        epochs=p['epochs'],
        verbose=0,
        validation_data=(x_validate, y_validate),
    )
    score = cnn_model.evaluate(x_test, y_test, verbose=0)
    print('===========================')
    print('>>>> Test Loss : {:.4f}'.format(score[0]))
    print('>>>> Test Accuracy : {:.4f}'.format(score[1]))
    print('===========================\n\n')
    cnn_model.reset_states()
    del cnn_model

>>> Params:  {'model': <keras.engine.sequential.Sequential object at 0x3f97d85b0>, 'adam_lr': 0.0025, 'batch_size': 64, 'epochs': 30}


2023-02-22 16:45:16.285451: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-22 16:45:21.766472: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


>>>> Test Loss : 0.2555
>>>> Test Accuracy : 0.9085


>>> Params:  {'model': <keras.engine.sequential.Sequential object at 0x46e56ec80>, 'adam_lr': 0.0025, 'batch_size': 128, 'epochs': 30}


2023-02-22 16:48:21.537224: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-22 16:48:25.935315: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


>>>> Test Loss : 0.2714
>>>> Test Accuracy : 0.9071


